In [1]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re

In [2]:
def mse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean() if reduction == "mean" else out 

In [3]:

p = '/data1/jessica/data/toyota/ckpts_final/'

experiments = os.listdir(p)
res = {}
res_ckpt = {}
for elem in experiments:
    path = p + elem + "/lightning_logs/" 
    vs = os.listdir(path)
    filt = []
    for elem1 in vs: 
        if 'version' in elem1:
            filt.append(elem1)
    versions =[elem.split("_")[-1]for elem in filt]
    versions = sorted(versions)
    version = f"version_{versions[-1]}"
    if elem == 'ckpts_final_comma_distance_clip' or elem == 'ckpts_final_comma_multitask_vit': version = "version_0"
    print(version, elem)

    checkpoint_path = path + version + "/checkpoints/"
    if "checkpoints" not in os.listdir(path + version): continue
    files = os.listdir(checkpoint_path)

    task = []
    ckpt = []
    files = sorted(files)
    for filename in files: 
        if filename.endswith(".csv"):
            df = pd.read_csv(checkpoint_path + filename)
            df.columns = ['preds', 'targets']
            m = (df['targets'] > 50).astype(bool) | (df['targets'] == 0).astype(bool) 
            
            loss3 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
            task.append(loss3.item())
        if filename.endswith(".ckpt"):
            ckpt.append(checkpoint_path + '/' + filename)
    res[elem] = task
    res_ckpt[elem] = ckpt


version_0 ckpts_final_comma_distance_resnet
version_0 ckpts_final_comma_multitask_none
version_6 ckpts_final_nuscenes_angle_vit
version_3 ckpts_final_nuscenes_multitask_none
version_5 ckpts_final_nuscenes_multitask_resnet
version_0 ckpts_final_comma_multitask_resnet
version_0 ckpts_final_comma_angle_resnet
version_8 ckpts_final_nuscenes_multitask_vit
version_4 ckpts_final_nuscenes_angle_resnet
version_7 ckpts_final_nuscenes_distance_vit
version_5 ckpts_final_nuscenes_multitask_clip
version_6 ckpts_final_nuscenes_angle_none
version_0 ckpts_final_comma_distance_none
version_5 ckpts_final_nuscenes_angle_clip
version_3 ckpts_final_nuscenes_distance_clip
version_0 ckpts_final_comma_multitask_vit
version_1 ckpts_final_comma_multitask_clip
version_0 ckpts_final_comma_distance_clip
version_7 ckpts_final_nuscenes_distance_none
version_0 ckpts_final_comma_angle_none
version_10 ckpts_final_comma_angle_vit
version_3 ckpts_final_nuscenes_distance_resnet
version_6 ckpts_final_comma_angle_clip
versio

In [4]:
os.listdir("/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_clip/lightning_logs")

['version_0']

In [5]:
res

{'ckpts_final_comma_distance_resnet': [4.665587373198741],
 'ckpts_final_comma_multitask_none': [4.579165410378686, 3.3693716510452907],
 'ckpts_final_nuscenes_angle_vit': [20.47928652916316],
 'ckpts_final_nuscenes_multitask_none': [16.87235935763188, 7.445182300121568],
 'ckpts_final_nuscenes_multitask_resnet': [],
 'ckpts_final_comma_multitask_resnet': [2.832363118532193, 4.987273033355811],
 'ckpts_final_comma_angle_resnet': [3.1649969769178874],
 'ckpts_final_nuscenes_angle_resnet': [],
 'ckpts_final_nuscenes_multitask_clip': [20.102755344247655,
  10.741788871675851],
 'ckpts_final_nuscenes_angle_none': [18.00808907106544],
 'ckpts_final_comma_distance_none': [3.3851383559744126],
 'ckpts_final_nuscenes_angle_clip': [16.028169846351133],
 'ckpts_final_nuscenes_distance_clip': [],
 'ckpts_final_comma_multitask_vit': [2.9565217244374127, 4.923400746465994],
 'ckpts_final_comma_multitask_clip': [],
 'ckpts_final_comma_distance_clip': [5.548644782425903],
 'ckpts_final_nuscenes_dista

In [6]:
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    data = splitted[2]
    task = splitted[3]
    backbone = splitted[4]
    rows.append({"Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": res[elem][0] if len(res[elem]) == 1 else res[elem]})



In [7]:
rows

[{'Task': 'distance',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 4.665587373198741},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'comma',
  't-RMSE': [4.579165410378686, 3.3693716510452907]},
 {'Task': 'angle',
  'Backbone': 'vit',
  'Dataset': 'nuscenes',
  't-RMSE': 20.47928652916316},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'nuscenes',
  't-RMSE': [16.87235935763188, 7.445182300121568]},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'nuscenes',
  't-RMSE': []},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': [2.832363118532193, 4.987273033355811]},
 {'Task': 'angle',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 3.1649969769178874},
 {'Task': 'angle', 'Backbone': 'resnet', 'Dataset': 'nuscenes', 't-RMSE': []},
 {'Task': 'multitask',
  'Backbone': 'clip',
  'Dataset': 'nuscenes',
  't-RMSE': [20.102755344247655, 10.741788871675851]},
 {'Task': 'angle',
  'Backbone': 'none',
  'Da

In [8]:
df = pd.DataFrame(rows)
df

,Task,Backbone,Dataset,t-RMSE
0,distance,resnet,comma,4.665587
1,multitask,none,comma,"[4.579165410378686, 3.3693716510452907]"
2,angle,vit,nuscenes,20.479287
3,multitask,none,nuscenes,"[16.87235935763188, 7.445182300121568]"
4,multitask,resnet,nuscenes,[]
5,multitask,resnet,comma,"[2.832363118532193, 4.987273033355811]"
6,angle,resnet,comma,3.164997
7,angle,resnet,nuscenes,[]
8,multitask,clip,nuscenes,"[20.102755344247655, 10.741788871675851]"
9,angle,none,nuscenes,18.008089


In [9]:


# The list of dictionaries
data = rows

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = ['t-RMSE ' + col for col in df_pivot.columns]

# Reset the index
df_pivot = df_pivot.reset_index()

# Display the pivoted DataFrame
print(df_pivot.to_latex())


\begin{tabular}{llllll}
\toprule
{} &   Dataset & Backbone & t-RMSE angle & t-RMSE distance &                          t-RMSE multitask \\
\midrule
0 &     comma &     clip &     2.122526 &        5.548645 &                                        [] \\
1 &     comma &     none &     5.155869 &        3.385138 &   [4.579165410378686, 3.3693716510452907] \\
2 &     comma &   resnet &     3.164997 &        4.665587 &    [2.832363118532193, 4.987273033355811] \\
3 &     comma &      vit &           [] &        5.023063 &   [2.9565217244374127, 4.923400746465994] \\
4 &  nuscenes &     clip &     16.02817 &              [] &  [20.102755344247655, 10.741788871675851] \\
5 &  nuscenes &     none &    18.008089 &        7.708046 &    [16.87235935763188, 7.445182300121568] \\
6 &  nuscenes &   resnet &           [] &              [] &                                        [] \\
7 &  nuscenes &      vit &    20.479287 &             NaN &                                       NaN \\
\bottomrule


/tmp/ipykernel_2190665/1832172679.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex())


In [10]:
df_pivot

,Dataset,Backbone,t-RMSE angle,t-RMSE distance,t-RMSE multitask
0,comma,clip,2.122526,5.548645,[]
1,comma,none,5.155869,3.385138,"[4.579165410378686, 3.3693716510452907]"
2,comma,resnet,3.164997,4.665587,"[2.832363118532193, 4.987273033355811]"
3,comma,vit,[],5.023063,"[2.9565217244374127, 4.923400746465994]"
4,nuscenes,clip,16.02817,[],"[20.102755344247655, 10.741788871675851]"
5,nuscenes,none,18.008089,7.708046,"[16.87235935763188, 7.445182300121568]"
6,nuscenes,resnet,[],[],[]
7,nuscenes,vit,20.479287,NaN,NaN
